## 1 加载包

In [1]:
library(stm)
library(tidyverse)
library(jiebaR)
library(quanteda)
library(showtext)
library(showtextdb)
# 自动使用 showtext 渲染
showtext_auto()

stm v1.3.7 successfully loaded. See ?stm for help. 
 Papers, resources, and other materials at structuraltopicmodel.com



── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     


── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: jiebaRD



Package version: 4.0.2
Unicode version: 14.0
ICU version: 70.1



Parallel computing: 4 of 4 threads used.



See https://quanteda.io for tutorials and examples.



Loading required package: sysfonts



Loading required package: showtextdb



## 2 整理数据

In [2]:
#导入数据
data1 <- read.csv(file = "/kaggle/input/dou-data1007/douban_data.csv")


In [3]:
glimpse(data1)

Rows: 108,985
Columns: 10
$ 标题          <chr> "# 阿尔茨海默病：一份温馨的护理指南", ".", ".", "/", "/"…
$ year          <dbl> 2024, 2023, 2023, 2021, 2021, 2021, 2021, 2021, 2021, 20…
$ ip            <chr> "上海", "北京", "海南", "", "", "", "", "", "", "", "", …
$ text          <chr> "# 阿尔茨海默病：一份温馨的护理指南\n      #阿尔茨海默病…
$ diqu          <dbl> 1, 1, 1, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,…
$ zuwu          <int> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ text1         <chr> "阿尔茨海默病一份温馨的护理指南阿尔茨海默病一份温馨的护…
$ text1_cut     <chr> "病 一份 护理 指南 病 一份 护理 指南 豆瓣 充满 文艺 气息…
$ text1_cleaned <chr> "病 一份 护理 指南 病 一份 护理 指南 豆瓣 充满 文艺 气息…
$ text2_cleaned <chr> "病 一份 护理 指南 病 一份 护理 指南 充满 气息 社区 分享…


In [4]:
# 从 data1 中选择指定列，并按标题、year 和 ip 合并相同的行，最后重命名列名
data <- data1 %>%
  select(标题, year, diqu, text2_cleaned) %>%
  group_by(标题, year, diqu) %>%
  summarise(
    text = paste(text2_cleaned, collapse = " "),  # 合并 text2_cleaned 并重命名为 text
    .groups = "drop"  # 解除分组
  )


In [5]:
glimpse(data)

Rows: 9,253
Columns: 4
$ 标题 <chr> "# 阿尔茨海默病：一份温馨的护理指南", ".", "/", "0", "00后提前经…
$ year <dbl> 2024, 2023, 2021, 2021, 2022, 2023, 2024, 2024, 2021, 2020, 2021,…
$ diqu <dbl> 1, 1, NA, NA, NA, 1, 1, 2, NA, NA, NA, 1, 2, 3, NA, 1, 2, 1, 2, 4…
$ text <chr> "病 一份 护理 指南 病 一份 护理 指南 充满 气息 社区 分享 感悟 知…


In [6]:
table(data$year,useNA="ifany")


2019 2020 2021 2022 2023 2024  NaN <NA> 
  41  548 1880 2579 2667 1506   30    2 

In [7]:
# 过滤掉 year 列中为 NaN 或 NA 的行
data <- data %>%
  filter(!is.na(year) & !is.nan(year))

# 检查结果
table(data$year, useNA = "ifany")


2019 2020 2021 2022 2023 2024 
  41  548 1880 2579 2667 1506 

In [8]:
table(data$diqu, useNA = "ifany")


   1    2    3    4 <NA> 
2436 1914  532  458 3881 

## 3 准备数据

In [9]:
# 4. Text processing using textProcessor (since the text is already segmented)
processed <- textProcessor(documents = data$text, 
                           metadata = data, 
                           removestopwords = FALSE,
                           removepunctuation = FALSE,
                           stem = FALSE,
                          wordLengths = c(2,Inf))



Building corpus... 
Converting to Lower Case... 
Removing numbers... 
Creating Output... 


In [10]:
# 5. Prepare documents for STM without removing low frequency terms
out <- prepDocuments(processed$documents, processed$vocab, processed$meta)
print(head(out$documents))  # View the first few processed documents

Removing 170 of 12986 terms (170 of 482905 tokens) due to frequency 
Your corpus now has 9026 documents, 12816 terms and 482735 tokens.

$`1`
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]   45  149  192  245  654  918 1054 1238 1320  1460  1478  1539  1629  1775
[2,]    2    2    2    1    2    1    1    1    1     1     1     1     1     1
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]  1867  2241  2289  2397  2474  2535  2759  2953  2987  3332  3719  3724
[2,]     2     1     1     5     1     2     1     1     1     1     1     1
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]  3731  4594  4604  4641  4642  4704  4750  4855  4879  4948  5112  5114
[2,]     1     1     1     1     1     2     2     1     1     1     1     2
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]  5161  5320  5598  5616  5742  5841  5844  5855  5861  5866  5959  5978
[2,]     1     2     1     1     2     1    15     1     1     3     1     3
     [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,6

In [11]:
# out$documents格式为：每条文本分词对应词汇表vocab中的id，以及词频
head(out$documents, 5)
head(out$vocab, 200)
head(out$meta, 5)

$`1`
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]   45  149  192  245  654  918 1054 1238 1320  1460  1478  1539  1629  1775
[2,]    2    2    2    1    2    1    1    1    1     1     1     1     1     1
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]  1867  2241  2289  2397  2474  2535  2759  2953  2987  3332  3719  3724
[2,]     2     1     1     5     1     2     1     1     1     1     1     1
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]  3731  4594  4604  4641  4642  4704  4750  4855  4879  4948  5112  5114
[2,]     1     1     1     1     1     2     2     1     1     1     1     2
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]  5161  5320  5598  5616  5742  5841  5844  5855  5861  5866  5959  5978
[2,]     1     2     1     1     2     1    15     1     1     3     1     3
     [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60] [,61] [,62]
[1,]  5981  5988  6013  6321  6336  6392  6603  6810  6830  6868  7018  7119
[2,]     1     1     1     1     1    10     3     1     3     1     3     1
     [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71] [,72] [,73] [,74]
[1,]  7161  7168  7250  7253  7270  7350  7375  7528  7933  8002  8335  8532
[2,]     1     1     1     1     1     1     1     3     1     1     2     2
     [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83] [,84] [,85] [,86]
[1,]  8549  8552  8573  8649  8715  8758  8801  9051  9069  9390  9431  9472
[2,]     4     1     1     1     1     3     4     4     1     3     1     1
     [,87] [,88] [,89] [,90] [,91] [,92] [,93] [,94] [,95] [,96] [,97] [,98]
[1,]  9483  9514  9713  9802 10013 10026 10058 10294 10388 10660 10661 10847
[2,]     3     1     1     2     1     1     2     3     1     1     1     1
     [,99] [,100] [,101] [,102] [,103] [,104] [,105] [,106] [,107] [,108]
[1,] 10927  11047  11058  11114  11134  11138  11180  11182  11235  11555
[2,]     1      2      1      1      2      4      1      1      1      1
     [,109] [,110] [,111] [,112] [,113] [,114] [,115] [,116] [,117] [,118]
[1,]  11617  11794  11962  11984  11989  12311  12463  12495  12511  12806
[2,]      2      1      1      2      1      2      1      1      1      2

$`2`
     [,1] [,2]  [,3]  [,4]  [,5]  [,6]  [,7]
[1,] 5151 9246 10453 11183 11271 12231 12263
[2,]    2    1     2     1     1     1     1

$`3`
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]   42   76  108  109  128  163  169  205  227   238   286   312   370   391
[2,]    3    9    6    3    9    3    9    3    3     9     3     3     3     9
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]   407   446   488   502   512   526   540   561   591   606   620   624
[2,]     6     6     9    12     3     6     3     9     6     6     6    12
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35] [,36] [,37] [,38]
[1,]   635   679   703   708   711   736   738   745   843   849   864   867
[2,]     3     3     3     3    27     3    15     3     3     6     3     3
     [,39] [,40] [,41] [,42] [,43] [,44] [,45] [,46] [,47] [,48] [,49] [,50]
[1,]   912   941   950  1050  1068  1079  1110  1135  1219  1232  1247  1328
[2,]     3     3     3     3     9     6     3     3     3     3     6     3
     [,51] [,52] [,53] [,54] [,55] [,56] [,57] [,58] [,59] [,60] [,61] [,62]
[1,]  1338  1404  1461  1472  1474  1480  1517  1534  1542  1559  1628  1642
[2,]     3     6     3     3     3     6     3     3     3     3     3     3
     [,63] [,64] [,65] [,66] [,67] [,68] [,69] [,70] [,71] [,72] [,73] [,74]
[1,]  1797  1820  1822  1852  1865  1918  1995  2056  2063  2078  2137  2218
[2,]     3     3     3     3     3     3     6     3     3     3     9     3
     [,75] [,76] [,77] [,78] [,79] [,80] [,81] [,82] [,83] [,84] [,85] [,86]
[1,]  2247  2302  2305  2333  2359  2385  2409  2413  2415  2439  2542  

[1] "一丁点"   "一万"     "一万个"   "一万二"   "一万五"   "一万元"  
  [7] "一万块"   "一万多"   "一万左右" "一万步"   "一下下"   "一下子"  
 [13] "一丝"     "一两"     "一两万"   "一两个"   "一两千"   "一两句"  
 [19] "一两周"   "一两天"   "一两岁"   "一两年"   "一两次"   "一两点"  
 [25] "一两百"   "一两百万" "一个个"   "一个包"   "一个半"   "一个半月"
 [31] "一个多"   "一个多月" "一个愿打" "一个月"   "一个样"   "一个点"  
 [37] "一个班"   "一个男孩" "一事"     "一二"     "一代"     "一代人"  
 [43] "一代代"   "一件"     "一份"     "一会"     "一会儿"   "一位"    
 [49] "一体"     "一個"     "一倍"     "一儿"     "一出"     "一刀"    
 [55] "一分"     "一分一毫" "一分钟"   "一分钱"   "一刻"     "一副"    
 [61] "一包"     "一千"     "一千万"   "一千五"   "一千块"   "一千多"  
 [67] "一千左右" "一半"     "一半一半" "一半儿"   "一半多"   "一厅"    
 [73] "一双"     "一发"     "一口"     "一句"     "一只"     "一台"    
 [79] "一名"     "一员"     "一周"     "一味"     "一回"     "一团"    
 [85] "一圈"     "一场"     "一块"     "一块钱"   "一堆"     "一声"    
 [91] "一处"     "一多"     "一夜"     "一大"     "一大半"   "一大块"  
 [97] "一大堆"   "一大截"   "一大笔"   "一大部分" "一天两天" "一天天"  
[103] "一头"     "一套"     "一套套"   "一女"     "一室"     "一家"    
[109] "一家人"   "一家子"   "一对"     "一对一"   "一封"     "一小块"  
[115] "一小部分" "一层"     "一层楼"   "一岁"     "一带"     "一帮"    
[121] "一幕"     "一平"     "一年"     "一年半载" "一床"     "一座"    
[127] "一弟"     "一张"     "一户"     "一所"     "一手"     "一打"    
[133] "一批"     "一把"     "一招"     "一拳"     "一挺"     "一排"    
[139] "一搏"     "一摊"     "一支"     "一整天"   "一斤"     "一方"    
[145] "一日"     "一日三餐" "一晃"     "一月"     "一本"     "一朵"    
[151] "一条"     "一杯"     "一枚"     "一架"     "一栋"     "一根"    
[157] "一桌"     "一档"     "一棵"     "一楼"     "一次次"   "一款"    
[163] "一步"     "一步步"   "一段"     "一毛钱"   "一流"     "一滴"    
[169] "一点"     "一点一点" "一点儿"   "一点多"   "一点点"   "一瓶"    
[175] "一生"     "一百"     "一百万"   "一百个"   "一百块"   "一百多"  
[181] "一百多万" "一百多元" "一百多块" "一百岁"   "一盆"     "一盒"    
[187] "一盘"     "一眼"     "一瞬间"   "一碗"     "一碗水"   "一种"    
[193] "一秒"     "一程"     "一站"     "一笔"     "一等"     "一箱"    
[199] "一篇"     "一米"

标题                               year diqu
1 # 阿尔茨海默病：一份温馨的护理指南 2024  1  
2 .                                  2023  1  
3 /                                  2021 NA  
4 0                                  2021 NA  
5 00后提前经历养老                   2022 NA  
  text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

## 4 searchk

In [12]:
#另一个notebook

## 5 拟合几个

In [13]:
stm_20 <- stm(documents = out$documents, vocab = out$vocab, K = 20, prevalence = ~year, data = out$meta, max.em.its = 75, verbose = FALSE)
summary(stm_20)

A topic model with 20 topics, 9026 documents and a 12816 word dictionary.


Topic 1 Top Words:
 	 Highest Prob: 独立, 控制, 不想, 吵架, 回家, 一点, 自私 
 	 FREX: 搬出去, 不理, 反抗, 远离, 吵架, 逃离, 住在一起 
 	 Lift: 寿宴, 干仗, 开厂, 管钱, 德性, 远点, 低气压 
 	 Score: 搬出去, 独立, 吵架, 窒息, 自私, 远离, 控制 
Topic 2 Top Words:
 	 Highest Prob: 父母, 子女, 选择, 人生, 想法, 希望, 照顾 
 	 FREX: 父母, 出国, 子女, 不育, 养儿防老, 选择, 生子 
 	 Lift: 上该, 共产主义, 客体, 私事, 细语, 晚育, 迁到 
 	 Score: 父母, 子女, 出国, 人生, 不育, 不婚, 独立 
Topic 3 Top Words:
 	 Highest Prob: 回去, 回家, 退休, 地方, 回来, 感觉, 一线 
 	 FREX: 接到, 开车, 加班, 一线, 保洁, 回去, 家乡 
 	 Lift: 一两点, 三四点, 偷听, 写字楼, 冲进来, 含蓄, 听得见 
 	 Score: 回去, 一线, 退休, 回家, 保洁, 开车, 小区 
Topic 4 Top Words:
 	 Highest Prob: 抱抱, 加油, 希望, 人生, 活着, 事情, 朋友 
 	 FREX: 加油, 抱抱, 坚强, 活着, 落户, 珍惜, 形婚 
 	 Lift: 生得, 项目经理, 挂失, 形婚, 很丧, 沼泽, 好事儿 
 	 Score: 加油, 抱抱, 人生, 坚强, 户口, 形婚, 节哀 
Topic 5 Top Words:
 	 Highest Prob: 体检, 身体, 喝酒, 医院, 医生, 健康, 抽烟 
 	 FREX: 戒烟, 戒酒, 喝酒, 抽烟, 套餐, 血脂, 体检中心 
 	 Lift: 体检中心, 倒入, 催吐, 内分泌科, 减退, 助听器, 取掉 
 	 Score: 体检, 戒烟, 喝酒, 血压, 血糖, 高血压, 糖尿病 
Topic 6 Top Words:
 	 Highest Prob: 子女, 儿子, 女儿, 重男轻女, 弟弟, 小孩, 不想 
 	 FREX: 二胎, 男孩, 不生, 再生, 重男轻

In [14]:
effects_20 <- estimateEffect(1:20 ~ year, stm_20, meta = out$meta)
summary(effects_20)


Call:
estimateEffect(formula = 1:20 ~ year, stmobj = stm_20, metadata = out$meta)


Topic 1:

Coefficients:
             Estimate Std. Error t value Pr(>|t|)
(Intercept)  1.847038   2.068908   0.893    0.372
year        -0.000885   0.001023  -0.865    0.387


Topic 2:

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 18.251175   2.079378   8.777   <2e-16 ***
year        -0.008988   0.001028  -8.742   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Topic 3:

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  8.311423   2.404068   3.457 0.000548 ***
year        -0.004079   0.001189  -3.431 0.000604 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Topic 4:

Coefficients:
             Estimate Std. Error t value Pr(>|t|)
(Intercept)  2.573549   2.124872   1.211    0.226
year        -0.001248   0.001051  -1.188    0.235


Topic 5:

Coefficients:
              Estimate Std. E

In [15]:
stm_10 <- stm(documents = out$documents, vocab = out$vocab, K = 10, prevalence = ~year, data = out$meta, max.em.its = 75, verbose = FALSE)
summary(stm_10)

A topic model with 10 topics, 9026 documents and a 12816 word dictionary.


Topic 1 Top Words:
 	 Highest Prob: 事情, 不想, 家庭, 朋友, 人生, 独生子女, 开心 
 	 FREX: 人生, 开心, 说话, 烦恼, 不耐烦, 事情, 人格 
 	 Lift: 强压, 炸毛, 反射, 枢纽, 语速, 贬义, 大嫂 
 	 Score: 家庭, 不想, 人生, 事情, 独生子女, 朋友, 独立 
Topic 2 Top Words:
 	 Highest Prob: 感觉, 退休, 时间, 亲戚, 经济, 上班, 情况 
 	 FREX: 红包, 上班, 洗衣机, 加班, 存钱, 退休, 失业 
 	 Lift: 假冒, 几线, 别动, 刷子, 半截, 好几件, 娇贵 
 	 Score: 退休, 上班, 衣服, 亲戚, 感觉, 经济, 焦虑 
Topic 3 Top Words:
 	 Highest Prob: 照顾, 回家, 抱抱, 加油, 办法, 家人, 生病 
 	 FREX: 加油, 抱抱, 陪陪, 照顾, 坚强, 辞职, 身心 
 	 Lift: 四周, 引流, 强健, 所欲, 撑下去, 整件事, 生得 
 	 Score: 照顾, 抱抱, 加油, 回家, 家人, 辞职, 护工 
Topic 4 Top Words:
 	 Highest Prob: 理解, 希望, 沟通, 父母, 方式, 原因, 关心 
 	 FREX: 公主, 姥爷, 关心, 理解, 反抗, 头发, 情感 
 	 Lift: 不办, 夫权, 接梗, 新娘, 染色, 移交, 舞台 
 	 Score: 理解, 父母, 关心, 方式, 希望, 公主, 姥姥 
Topic 5 Top Words:
 	 Highest Prob: 结婚, 房子, 买房, 家庭, 条件, 儿子, 男人 
 	 FREX: 彩礼, 男方, 女方, 扶贫, 冠姓, 买房, 倒贴 
 	 Lift: 冠姓, 三室, 三金, 下嫁, 个富, 买楼, 了不得 
 	 Score: 结婚, 买房, 房子, 男方, 儿子, 女方, 彩礼 
Topic 6 Top Words:
 	 Highest Prob: 离婚, 情绪, 关系, 奶奶, 感觉, 改变, 吵架 
 	 FREX: 证据, 继承人, 起诉, 法院, 心理医生, 离婚, 立遗嘱 
 	 Li

In [16]:
effects_10 <- estimateEffect(1:10 ~ year, stm_10, meta = out$meta)
summary(effects_10)


Call:
estimateEffect(formula = 1:10 ~ year, stmobj = stm_10, metadata = out$meta)


Topic 1:

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  7.986648   1.179047   6.774 1.33e-11 ***
year        -0.003918   0.000583  -6.720 1.93e-11 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Topic 2:

Coefficients:
             Estimate Std. Error t value Pr(>|t|)  
(Intercept)  2.620145   1.557147   1.683   0.0925 .
year        -0.001255   0.000770  -1.630   0.1032  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Topic 3:

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -14.245307   2.363793  -6.026 1.74e-09 ***
year          0.007099   0.001169   6.073 1.31e-09 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Topic 4:

Coefficients:
              Estimate Std. Error t value Pr(>|t|)
(Intercept)  2.4828626  1.6644558   1.492    0.136
year        -0.0011898  0.0

In [17]:
# 将所有当前环境中的对象保存到指定位置
save.image('/kaggle/working/stm_douban.RData')


In [18]:
# List to store results for each model and its estimated effects
stm_models <- list()
effect_summaries <- list()

# Loop through topic numbers from 10 to 19
for (k in 10:19) {
  # Run the STM model
  stm_model <- stm(documents = out$documents, 
                   vocab = out$vocab, 
                   K = k, 
                   prevalence = ~year, 
                   data = out$meta, 
                   max.em.its = 75, 
                   verbose = FALSE)
  
  # Store the STM model in the list
  stm_models[[paste0("K_", k)]] <- stm_model
  
  # Print summary of the STM model (optional, you can remove this if not needed)
  print(paste("Summary for STM model with K =", k))
  print(summary(stm_model))
  
  # Estimate effects
  effects <- estimateEffect(1:k ~ year, stm_model, meta = out$meta)
  
  # Store the estimated effects summary in the list
  effect_summaries[[paste0("K_", k)]] <- summary(effects)
  
  # Print summary of the estimated effects (optional, you can remove this if not needed)
  print(paste("Summary of effects for K =", k))
  print(summary(effects))
}

# Access the models or effect summaries later if needed
# e.g., effect_summaries[["K_15"]]


[1] "Summary for STM model with K = 10"
A topic model with 10 topics, 9026 documents and a 12816 word dictionary.
Topic 1 Top Words:
 	 Highest Prob: 事情, 不想, 家庭, 朋友, 人生, 独生子女, 开心 
 	 FREX: 人生, 开心, 说话, 烦恼, 不耐烦, 事情, 人格 
 	 Lift: 强压, 炸毛, 反射, 枢纽, 语速, 贬义, 大嫂 
 	 Score: 家庭, 不想, 人生, 事情, 独生子女, 朋友, 独立 
Topic 2 Top Words:
 	 Highest Prob: 感觉, 退休, 时间, 亲戚, 经济, 上班, 情况 
 	 FREX: 红包, 上班, 洗衣机, 加班, 存钱, 退休, 失业 
 	 Lift: 假冒, 几线, 别动, 刷子, 半截, 好几件, 娇贵 
 	 Score: 退休, 上班, 衣服, 亲戚, 感觉, 经济, 焦虑 
Topic 3 Top Words:
 	 Highest Prob: 照顾, 回家, 抱抱, 加油, 办法, 家人, 生病 
 	 FREX: 加油, 抱抱, 陪陪, 照顾, 坚强, 辞职, 身心 
 	 Lift: 四周, 引流, 强健, 所欲, 撑下去, 整件事, 生得 
 	 Score: 照顾, 抱抱, 加油, 回家, 家人, 辞职, 护工 
Topic 4 Top Words:
 	 Highest Prob: 理解, 希望, 沟通, 父母, 方式, 原因, 关心 
 	 FREX: 公主, 姥爷, 关心, 理解, 反抗, 头发, 情感 
 	 Lift: 不办, 夫权, 接梗, 新娘, 染色, 移交, 舞台 
 	 Score: 理解, 父母, 关心, 方式, 希望, 公主, 姥姥 
Topic 5 Top Words:
 	 Highest Prob: 结婚, 房子, 买房, 家庭, 条件, 儿子, 男人 
 	 FREX: 彩礼, 男方, 女方, 扶贫, 冠姓, 买房, 倒贴 
 	 Lift: 冠姓, 三室, 三金, 下嫁, 个富, 买楼, 了不得 
 	 Score: 结婚, 买房, 房子, 男方, 儿子, 女方, 

In [19]:
# 将所有当前环境中的对象保存到指定位置
save.image('/kaggle/working/stm_douban1.RData')